In [25]:
from movie_recommender import MovieRecommender
import timeit
import pandas as pd

In [26]:
started = timeit.default_timer()
movies_file = 'data/ml-10M100K/movies.dat'
ratings_file = 'data/ml-10M100K/ratings.dat'
movie_rec = MovieRecommender(movies_filename=movies_file, ratings_filename=ratings_file)
elapsed = timeit.default_timer() - started
print(f'Read dataframes in {elapsed}ms')

print(movie_rec.ratings_df['user_id'].nunique())

found pickle at c:\Users\john.french\movie_recs\data\pickled\movie_ratings.pkl
Read dataframes in 28.676943099999335ms
69878


In [49]:
# replace pipe character with space
movie_rec.movies_df['genres'] = movie_rec.movies_df['genres'].str.replace('|', ' ')
rating_counts = movie_rec.ratings_df.groupby('movie_id').agg('rating').count()
most_rated = rating_counts.sort_values(ascending=False).head(1000)
most_rated = pd.merge(most_rated, movie_rec.movies_df, on="movie_id")
most_rated

C:\Users\john.french\AppData\Local\Temp\ipykernel_21940\133245661.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  movie_rec.movies_df['genres'] = movie_rec.movies_df['genres'].str.replace('|', ' ')


,rating,name,genres
movie_id,,,
296,34864,Pulp Fiction (1994),Comedy Crime Drama
356,34457,Forrest Gump (1994),Comedy Drama Romance War
593,33668,"Silence of the Lambs, The (1991)",Crime Horror Thriller
480,32631,Jurassic Park (1993),Action Adventure Sci-Fi Thriller
318,31126,"Shawshank Redemption, The (1994)",Drama
...,...,...,...
2728,2577,Spartacus (1960),Action Drama Romance War
1956,2574,Ordinary People (1980),Drama
1344,2573,Cape Fear (1962),Thriller


In [46]:
ctr = 0
limit = 15
for idx, row in most_rated.iterrows():
    print(row['name'], idx)
    ctr += 1
    if ctr > limit:
        break

nan 297
Forrest Gump (1994) 356
Silence of the Lambs, The (1991) 593
Jurassic Park (1993) 480
Shawshank Redemption, The (1994) 318
Braveheart (1995) 110
Fugitive, The (1993) 457
Terminator 2: Judgment Day (1991) 589
Star Wars: Episode IV - A New Hope (a.k.a. Star Wars) (1977) 260
Apollo 13 (1995) 150


In [51]:
descriptions_df = pd.read_json('data/metadata/movie_descriptions.json')
most_rated = pd.merge(descriptions_df, most_rated, on='movie_id', how='right')
most_rated = most_rated.head(1000)
most_rated.set_index('movie_id', inplace=True)
most_rated.to_json('data/metadata/top_rated_1000.json', orient='index')

In [ ]:
from ai_posters import test_api_route

ctr = 0
offset = 300
limit = 500
for idx, row in most_rated.iterrows():
    movie_title = row['name']
    genres = row['genres']
    movie_id = idx
    ctr += 1
    if ctr >= offset:
        test_api_route.make_poster_for_movie(movie_title, genres, str(movie_id))
    if ctr == limit:
        break

In [ ]:
# clean_titles = [movie_rec.get_movie_title(m) for m in set(movie_rec.ratings_df.index.values)]
import pandas as pd

# create a DataFrame containing the movie titles and ids
movies_df = pd.DataFrame({'title': movie_rec.movies_df['name']})

# count the occurrences of each title using the value_counts() method
title_counts = movies_df['title'].value_counts()

# filter out the titles that appear only once using boolean indexing
popular_titles = title_counts[title_counts > 1].index.tolist()

# get the corresponding movie ids for the popular titles
popular_movie_ids = movies_df.index.get_level_values('movie_id')[movies_df['title'].isin(popular_titles)].unique().tolist()

print(popular_movie_ids)

In [ ]:
u = int(movie_rec.mrm_df.shape[0])
# u = 2
# print(type(u), u)

# movie_rec.new_add_movie_rating(movie_id=1219, user_id=u, rating=5) # Psycho
# How to make an american quilt
# movie_rec.new_add_movie_rating(movie_id=46, user_id=u, rating=5) 
# movie_rec.new_add_movie_rating(movie_id=16, user_id=u, rating=5) # Casino
# movie_rec.new_add_movie_rating(movie_id=2596, user_id=u, rating=5) # Rocky Horror
# movie_rec.new_add_movie_rating(movie_id=2571, user_id=u, rating=5) # The Matrix
movie_rec.new_add_movie_rating(movie_id=107, user_id=u, rating=5) # Muppet Treasure Island
movie_rec.new_add_movie_rating(movie_id=5080, user_id=u, rating=5) # Slackers
# movie_rec.new_add_movie_rating(movie_id=30, user_id=u, rating=5)
# movie_rec.new_add_movie_rating(movie_id=232, user_id=u, rating=5)
# movie_rec.new_add_movie_rating(movie_id=858, user_id=u, rating=5)
# movie_rec.new_add_movie_rating(movie_id=48, user_id=u, rating=5)
# movie_rec.new_add_movie_rating(movie_id=1213, user_id=u, rating=5)
# movie_rec.new_add_movie_rating(movie_id=858, user_id=u, rating=5)

movie_rec.refit_model()


In [ ]:
movie_rec.get_user_based_recommendation(u)

In [ ]:
# movie_rec.get_user_movies(69877)
for m in movie_rec.get_user_movies(69877):
    print(movie_rec.get_movie_title(m), movie_rec.get_user_movie_rating(69877, m))
# movie_rec.get_user_movie_rating(8546, 1)
# movie_rec.get_user_movie_rating(8546, 2)
# print(f'User {u} has seen...')
# for m in movie_rec.get_user_movies(u):
#     print(movie_rec.get_movie_title(m))

In [ ]:
users, dists = movie_rec.get_similar_users(u)
# for rec_user in users:
#     # print(f'{rec_user}\'s movies')
#     for m in movie_rec.get_user_movies(rec_user):
#         # print(movie_rec.get_movie_title(m))

# print(movie_rec.get_movie_title(20))
# print(movie_rec.get_movie_title(25))
# print(movie_rec.get_movie_title(50))

In [ ]:
user_id = movie_rec.get_next_avail_user_id()
# movie_rec.add_movie_rating(movie_id=1, user_id= user_id, rating= 5) # Toy Story
# movie_rec.add_movie_rating(movie_id=2, user_id= user_id, rating= 5) # Jumanji
# movie_rec.add_movie_rating(movie_id=48, user_id= user_id, rating= 5) # Pocahontas
# movie_rec.add_movie_rating(movie_id=20, user_id= user_id, rating= 5) # Money Train
# movie_rec.add_movie_rating(movie_id=25, user_id= user_id, rating= 5) # Leaving Las Vegas
# movie_rec.add_movie_rating(movie_id=50, user_id= user_id, rating= 5) # The Usual Suspects
movie_rec.ratings_df.head()

In [ ]:
import numpy as np


# return the difference between average ratings for two movies
def get_movie_mean_rating_error(movie1_id: int, movie2_id: int):
    movie1_ratings = np.mean(
        movie_rec.ratings_df.loc[movie_rec.ratings_df.index == movie1_id]['rating'].values)
    movie2_ratings = np.mean(
        movie_rec.ratings_df.loc[movie_rec.ratings_df.index == movie2_id]['rating'].values)
    return abs(movie1_ratings - movie2_ratings)


# get_movie_mean_rating_error(599, 700)
# movie_rec.get_movie_users(5)
print(movie_rec.get_movie_title(1))
print(movie_rec.get_movie_title(2))
print(len(movie_rec.get_movie_users(1)))
print(len(movie_rec.get_movie_users(2)))
print(len(movie_rec.get_common_users(1, 2)))

In [24]:
def get_movie_rating_pearson_coeff(movie1_id: int, movie2_id: int)-> float:
    common_users = movie_rec.get_common_users(movie1_id=movie1_id, movie2_id=movie2_id)
    movie1_ratings = [movie_rec.get_user_movie_rating(user_id=u, movie_id=movie1_id) for u in common_users]
    movie2_ratings = [movie_rec.get_user_movie_rating(user_id=u, movie_id=movie2_id) for u in common_users]
    coeff = np.corrcoef(np.array(movie1_ratings), np.array(movie2_ratings))[0, 1]
    movie1_title = movie_rec.get_movie_title(movie1_id)
    movie2_title = movie_rec.get_movie_title(movie2_id)
    print(f'{movie1_title} and {movie2_title} have a correlation of {"{:.2f}".format(coeff)} from {len(common_users)} users who have rated both')

get_movie_rating_pearson_coeff(1, 3)

KeyboardInterrupt: 